- Cleaning and preparing data;
- Building feature vectors from text documents;
- Training a machine learning model to classify positive and negative movie reviews;
- Working with large text datasets using out-of-core learning;
- Inferring topics from document collections for categorization.

# Preparing the IMDb movie review data for text processing
Dataset can be found [here](https://ai.stanford.edu/~amaas/data/sentiment/).

Now let's preprocess the dataset.

In [13]:
# import pyprind
# import pandas as pd
# import os
# import sys

# basepath='aclImdb'
# labels={'pos':1,'neg':0}
# pbar = pyprind.ProgBar(50000,stream=sys.stdout)
# df = pd.DataFrame()
# for s in ('test','train'):
#     for l in ('pos','neg'):
#         path = os.path.join(basepath,s,l)
#         for file in sorted(os.listdir(path)):
#             with open(os.path.join(path,file),'r',encoding='utf-8') as infile:
#                 txt = infile.read()
#             df = pd.concat([df,pd.DataFrame([[txt,labels[l]]])],ignore_index=True)
#             pbar.update()
# df.columns = ['review','sentiment']

In [14]:
# Let's save it in a more handy format
# import numpy as np
# np.random.seed(0)
# df = df.reindex(np.random.permutation(df.index))
# df.to_csv('movie_data.csv',index=False,encoding='utf-8')

In [15]:
import numpy as np
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
print(df.shape)
df.head(3)

(50000, 2)


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


## Introducing the bag-of-words model

1. We create a vocabulary of unique tokens (for example words) from the entire set of documents;
2. We construct a feature vector from each document that contains the counts of how often each word occurs in the particular document.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining','The weather is sweet','The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())
# NOTE: the word or term order in a sentence doesn't matter.
# NOTE: right now we are using a 1-gram representation. If we want to change it we just have to modify the `ngram_range` parameter

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


## Assessing word relevancy via term frequency-inverse document frequency (TF-IDF)

There are frequently occurring words that typically don't contain useful or discriminatory information. TF-IDF can be used to downweight these frequently occuring words in the feature vectors.
$$tf-idf(t,d) = tf(t,d)\times idf(t,d)$$
where the tf is the term frequency introduced in the previous section, while the idf is:
$$idf(t,d) = \log(\frac{n_d}{1+df(d,t)})$$
Here $n_d$ is the total number of documents, and $df(d,t)$$ it's the number of documents $d$ that contains the term $t$.

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


The tf-idf formula works a little bit differenty compared to what showed before:
- The `smooth_idf` add a $+1$ to certain parts of the equation in order to assign a weight of 0 from the idf formula if one elements compares in every document, while adding a $+1$ to the idf result in order to not make the term totally unsignificant;
- The `norm` applies the selected normalization, returning a vector of length 1, to the term frequency vector after the tf-idf calculation.

Let's now clean the reviews from unwanted characters.

In [22]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>','',text) # tried to remove all the HTML markup
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text) # finding all the emoticons
    text = (
        re.sub('[\W]+',' ',text.lower()) + # removed all non-word characters from the text
        ' '.join(emoticons) # added found emoticons at the end of the screen
            .replace('-','') # removed the nose from faces for consistency
    )
    return text

df['review'] = df['review'].apply(preprocessor)

<>:4: SyntaxWarning: invalid escape sequence '\)'
<>:6: SyntaxWarning: invalid escape sequence '\W'
<>:4: SyntaxWarning: invalid escape sequence '\)'
<>:6: SyntaxWarning: invalid escape sequence '\W'
/var/folders/8q/dn061sy56s7b4yfppcpnhkqc0000gn/T/ipykernel_31432/1014577360.py:4: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text) # finding all the emoticons
/var/folders/8q/dn061sy56s7b4yfppcpnhkqc0000gn/T/ipykernel_31432/1014577360.py:6: SyntaxWarning: invalid escape sequence '\W'
  re.sub('[\W]+',' ',text.lower()) + # removed all non-word characters from the text


## Processing docuents into tokens

One way could be to split the cleaned docuents at their whitespace characters using something like `text.split()`.

However another useful technique is word stemming, which is the process of transforming a word into its root form.

In [27]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run') # notice: running -> run

import nltk

nltk.download('stopwords') # downloaded a list of english stopwords (to be removed because they are insignificant)

from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/riccardotoniolo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Training a logistic regrression model for document classification

In [25]:
X_train = df.loc[:25000,'review'].values
y_train = df.loc[:25000,'sentiment'].values
X_test = df.loc[25000:,'review'].values
y_test = df.loc[25000:,'sentiment'].values

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
# Note that TfidfVectorizer combines the CountVectorizer and the TfidfTransformer
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0,10.0]
    },
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0,10.0]
    },
]

lr_tfidf = Pipeline([['vect',tfidf],['clf',LogisticRegression(solver='liblinear')]])
# Usig liblinear instead of lbfgs since the former perform better than the latter in relatively large datasets
gs_lr_tfidf = GridSearchCV(lr_tfidf,small_param_grid,scoring="accuracy",cv=5,verbose=2,n_jobs=-1)
gs_lr_tfidf.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521

[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x1049f84a0>; total time=   2.7s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x1057784a0>; total time=   2.8s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x10310c4a0>; total time=   2.8s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x107df84a0>; total time=   2.8s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x10308c4a0>; total time=   2.7s


/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x1296a45e0>; total time=   3.0s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x10e4845e0>; total time=   3.1s


...

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 

/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 

/Users/riccardotoniolo/miniconda3/envs/mllib/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[['vect',
                                        TfidfVectorizer(lowercase=False)],
                                       ['clf',
                                        LogisticRegression(solver='liblinear')]]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x17e3ea020>,
                                              <function tokenizer_porter at 0x17e3e9c60>]},
                         {'...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x17e3ea020>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [31]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x17e3ea020>}


In [32]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test,y_test):.3f}')

CV Accuracy: 0.897
Test Accuracy: 0.899


# Working with bigger data -- online algorithms and out-of-core learning

It is not uncommon to work with even larger datasets that can exceed our computer's memory.

**Out-of-core learning** allows us to work with such large datasets by fitting the classifier incrementally on smaller batches of a dataset.

We will make use of the `partial_fit` function of `SGDClassifier` to stream the documents directly from our local drive and train a logistic regression model using small mini-batches of documents.

In [33]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>','',text) # tried to remove all the HTML markup
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text) # finding all the emoticons
    text = (
        re.sub('[\W]+',' ',text.lower()) + # removed all non-word characters from the text
        ' '.join(emoticons) # added found emoticons at the end of the screen
            .replace('-','') # removed the nose from faces for consistency
    )
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:9: SyntaxWarning: invalid escape sequence '\W'
<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:9: SyntaxWarning: invalid escape sequence '\W'
/var/folders/8q/dn061sy56s7b4yfppcpnhkqc0000gn/T/ipykernel_31432/3869798066.py:7: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text) # finding all the emoticons
/var/folders/8q/dn061sy56s7b4yfppcpnhkqc0000gn/T/ipykernel_31432/3869798066.py:9: SyntaxWarning: invalid escape sequence '\W'
  re.sub('[\W]+',' ',text.lower()) + # removed all non-word characters from the text


In [34]:
def stream_docs(path):
    with open(path,'r',encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text,label = line[:-3],int(line[-2])
            yield text, label

next(stream_docs(path="movie_data.csv"))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

We can't use CountVectorizer nor TfidfVectorizer, since they need all of the corpus to operate.

However anothe useful vectorizer for text processing is the HashingVectorizer, since is data-independent and make use of the hashing trick.

In [36]:
def get_minibatch(doc_stream,size):
    docs, y = [] ,[]
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [38]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=tokenizer)
# The higher the number of feature the lower the chance of hash collision, while also increasing the number
# of coefficients in the logistic regression model.
clf = SGDClassifier(loss="log_loss",random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

In [39]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
for _ in range(45):
    X_train,y_train = get_minibatch(doc_stream,size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train,y_train,classes=classes)
    pbar.update()

In [40]:
X_test, y_test = get_minibatch(doc_stream,size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test,y_test):.3f}')

Accuracy: 0.868


Observe that this kind of strategy it's really memory efficient, faster to run thus resulting in a slightly worse accuracy.

Another interesting alternative to the bag-of-words model is the word2vec model, that tries to put words that have similar meaning into similar clusters.

# Topic modeling with latent Dirichlet allocation

Topic modeling describes the broad task of assigning topics to unlabeled text documents. We can consider topic modeling as a clustering task, a subcategory of unsupervised learning.

Latent Dirichlet Allocation (LDA, not to be confused with *linear discriminant analysis*) is a popular technique for topic modeling.

## Decomposing text documents with LDA

LDA tries to find groups of words that appear frequently together across different documents. The input to an LDA is the bag-of-words model that we discussed earlier in this chapter.

Given a BoW matrix as input, LDA decomposes it into two new matrices:
- A document-to-topic matrix;
- A word-to-topic matrix.

LDA works in a way that if we multiply those two obtained matrices together, we will re-obtain the BoW original matrix.

The only downside is that we have to define a number of topics beforehand (an hyperparameter to the LDA algorithm).

## LDA with scikit-learn

In [41]:
import pandas as pd
df = pd.read_csv('movie_data.csv',encoding='utf-8')

from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(
    stop_words='english',
    max_df=.1, # Means that the maximum document frequency is 10% (words with a df above that will be ignored)
    max_features=5000 # The number of words will be a maximum of 5000 (the top 5000 most frequent)
)
# This was done to improve the inference speed of the LDA algorithm

X = count.fit_transform(df['review'].values)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,random_state=123,learning_method='batch',n_jobs=-1)
# batch means that all the training data is considered as a single batch
# the alternative is "online" but leads to worse results
X_topics = lda.fit_transform(X)

In [43]:
lda.components_.shape

(10, 5000)

In [ ]:
# Let's print the five most important words for each of the 10 topic
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx+1)}:')
    print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))
    # We are sorting the array in reverse order since originally the words are sorted in increasing order

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children kids
Topic 3:
american war dvd music black
Topic 4:
human audience art cinema feel
Topic 5:
police guy car dead murder
Topic 6:
horror house sex blood woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode episodes tv season
Topic 9:
book version original effects fi
Topic 10:
action fight guy guys cool


In [45]:
horror = X_topics[:,5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nHorror movie #{(iter_idx+1)}:')
    print(df['review'][movie_idx][:300],'...')


Horror movie #1:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...

Horror movie #2:
Before I talk about the ending of this film I will talk about the plot. Some dude named Gerald breaks his engagement to Kitty and runs off to Craven Castle in Scotland. After several months Kitty and her aunt venture off to Scottland. Arriving at Craven Castle Kitty finds that Gerald has aged and he ...

Horror movie #3:
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la ...
